## 1. Framework of RDAnalyzer
The framework of RDAnalyzer is shown in Figure 1. 
It contains four files that named as `set.py`, `tools.py`, `species.py`, and `analyzer.py`. 

The `set.py` contains settings of the RDAnalyzer toolkit, including atom types to distinguish atoms involved in the system (atomType), the bond order to decide whether there is a bond (bondOrder, default is 0.3), as well as the time step of ReaxFF MD simulations (timeStep) et, al. Before using RDAnalyzer, these settings need to be checked. 

The `tools.py` contains the various tools, for example, methods to deal with the periodic boundary condition (pbc), list2CSV to save data as csv files, and list2Fig to draw the data as figures et al.

The `Species.py` contains Atom, Molecule, and Reaction, which are used to save the atomic, molecular, and reaction information, respectively. Moreover, these classes are easy to be manipulated. 

The `analyzer.py` file contains methods to analyze the ReaxFF MD results to obtain the hydroxide diffusion mechanism and conductivity. 

<img src="./Tutorial_figs/1.png" width="50%"> \
Figure 1. The framework diagram of RDAnalyzer

In [ ]:
from RDAnalyzer.analyzer import Analyzer
from RDAnalyzer.tools import dump, load, printf, timer, moveFiles
from RDAnalyzer.species import Mol
import os

## 2. Setting up



- `DEFAULT_BONDER_ORDER = 0.5` \
The bond order that determines whether a bond is formed. \
The default value in LAMMPS is 0.3.\
The default value in RDAnalyzer is 0.5 for ensuring the existence of H3O2- and H5O3- etc.

- `ATOMTYPE = { 1: 'C', 2: 'H', 3: 'O', 4: 'N'}` \
Atom types in LAMMPS are number (1,2,3, etc.). \
RDAnalyzer needs the mapping dictionary  

- `TIMESTEP = 0.0001` \
time step (TIME_UNIT)

- `TIME_UNIT = 'ps'` \
time unit

- `DUMP_INTERVAL = 1000` \
dump interval of MD simulations

## 3. Splitting trajectory
ReaxFF MD trajectory may contain frames we do not care, so we need to split the trajectory as shown in Figure 2 

<img src="./Tutorial_figs/2.png" width="50%"> \
Figure 2. Schematic illustration of splitting ReaxFF MD trajectory

In [1]:
from RDAnalyzer.analyzer import Analyzer

# split txt
trjSplitted, bondSplitted = Analyzer.split(
    interval=10,
    beginStep=5000000,
    endStep=6000000,
    trjName="dump.trj",
    bondName="bonds.reax",
)

# The actual interval is `interval`*`DUMP_INTERVAL` = 10000 steps (1 ps)

#[split              ] Number of atoms 2450
#[split              ] Split dump.trj to dump.trj.pre10fra.5000000-6000000 done
#[split              ] Split bonds.reax to bonds.reax.pre10fra.5000000-6000000 done


## 4. Convert trajectory to Atom
Then we need to convert the trajectory (text) to `Atom`


<img src="./Tutorial_figs/3.png" width="50%"> \
Figure 3. Schematic illustration of transfer trajectory to `Atom` 

In [2]:
from RDAnalyzer.analyzer import Analyzer
from RDAnalyzer.tools import dump, printf
import os

# Convert txt to Atom
if os.path.exists('atmDict.pkl'): 
    printf("Exist atmDict.pkl")
else:
    atmDict = Analyzer.toAtoms(trjSplitted, bondSplitted)
    dump(atmDict, 'atmDict.pkl')

#[toAtoms            ] Begin convert txt to Atom
#[toAtoms            ] Convert txt to Atom done


## 5. Convert Atom to Molecule
Next we need to convert `Atom` to `Molecule`


<img src="./Tutorial_figs/4.png" width="50%"> \
Figure 4. Schematic illustration of transfer `Atom`  to `Molecule` 

In [3]:
from RDAnalyzer.analyzer import Analyzer
from RDAnalyzer.tools import dump, printf, load
import os

# Convert Atom to Molecule
if os.path.exists('molDict.pkl'):
    printf("Exist molDict.pkl")
else:
    atmDict = load('atmDict.pkl')
    molDict = Analyzer.toMolecules(atmDict)
    dump(molDict, 'molDict.pkl')

#[toMolecules        ] Begin convert Atom to Molecule
#[toMolecules        ] Convert Atom to Molecule done


## 6. Convert Molecule to Reaction
Next we need to convert `Molecule` to `Reaction`


<img src="./Tutorial_figs/5.png" width="50%"> \
Figure 5. Schematic illustration of transfer `Molecule`  to `Reaction` 

In [4]:
if os.path.exists('rexDict.pkl'):
    printf("Exist rexDict.pkl")
else:
    molDict = load("molDict.pkl")
    rexDict = Analyzer.toReactions(molDict, nCores=8)
    dump(rexDict, 'rexDict.pkl')

#[toReactions        ] Begin convert Molecule to Reaction
#[toReactions        ] Convert Molecule to Reaction done


## 7. Calculation of RDF and CN
RDF and CN are important for exploring hydroxide distribution in pores of AEMs, which can be obtained by RDAnalyzer

In [5]:
from RDAnalyzer.tools import dump, moveFiles
''' get RDF and CN by atomtype pair'''

if os.path.exists("RDF-CN"):
    printf("Exist RDF-CN")
else:
    molDict = load("molDict.pkl")
    stepList = [step for step in molDict.keys()]
    stepList = sorted(stepList)[-5:]
    Analyzer.getAtomAndAtomRDF(molDict=molDict, pair=(4, 3), step=stepList, r=11, n=100, smth=True)
    moveFiles(toDir="RDF-CN")
# 

#[listToCSV          ] store as: raw-RDF-CN-of-N-O-stp5960000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-CN-of-N-O-stp5970000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-CN-of-N-O-stp5980000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-CN-of-N-O-stp5990000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-CN-of-N-O-stp6000000-r11-n100.csv
#[drawFigMulti       ] store as: fig-RDF-of-N-O-step5960000-6000000-5-r11-n100.png
#[drawFigMulti       ] store as: fig-smthRDF-N-O-step5960000-6000000-5-r11-n100.png
#[listToCSV          ] store as: raw-smth-RDF-CN-of-N-O-stp5960000-6000000-5-r11-n100.csv
#[drawFigMulti       ] store as: fig-CN-of-N-O-step5960000-6000000-5-r11-n100.png
#[listToCSV          ] store as: raw-avg-RDF-CN-of-N-O-stp5960000-6000000-5-r11-n100.csv
#[getAtomAndAtomRDF  ] peak RDF: ( 3.8499999999999988 2.346261760925511 )
#[getAtomAndAtomRDF  ] in CN(r): ( 3.8499999999999988 3.4 )
#[moveFiles          ] moveFile: ['fig-CN-of-N-O-step5960000-60

We usually concern the RDF/CN about N-O_OH and N-O_W,
which can be calculated by getAtomAndAtomRDF_NO()

<img src="./Tutorial_figs/6.png" width="40%"> \
Figure 6. RDF(N-O) and CN(N-O*)

In [6]:
from RDAnalyzer.tools import dump, moveFiles
''' get RDF and CN of N-O'''

if os.path.exists("RDF-CN-special"):
    printf("Exist RDF-CN-special")
else:
    molDict = load("molDict.pkl")
    stepList = [step for step in molDict.keys()]
    stepList = sorted(stepList)[-10:]
    Analyzer.getAtomAndAtomRDF_NO(molDict=molDict, step=stepList, r=11, n=100, smth=True)
    moveFiles(toDir="RDF-CN-special")

#[getAtomAndAtomRDF  ] the Bias:  < (bigger than true value)  0.11
#[listToCSV          ] store as: raw-RDF-N_O-of-N-O-stp5910000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-N_O_OH-of-N-O_OH-stp5910000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-N_O_W-of-N-O_W-stp5910000-r11-n100.csv
#[getAtomAndAtomRDF  ] the Bias:  < (bigger than true value)  0.11
#[listToCSV          ] store as: raw-RDF-N_O-of-N-O-stp5920000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-N_O_OH-of-N-O_OH-stp5920000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-N_O_W-of-N-O_W-stp5920000-r11-n100.csv
#[getAtomAndAtomRDF  ] the Bias:  < (bigger than true value)  0.11
#[listToCSV          ] store as: raw-RDF-N_O-of-N-O-stp5930000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-N_O_OH-of-N-O_OH-stp5930000-r11-n100.csv
#[listToCSV          ] store as: raw-RDF-N_O_W-of-N-O_W-stp5930000-r11-n100.csv
#[getAtomAndAtomRDF  ] the Bias:  < (bigger than true value)  0.11
#[listToCSV       

## 8. Species, Reaction, and Reaction network

<img src="./Tutorial_figs/7.png" width="50%"> \
Figure 7. Species and Reaction Number

In [7]:
'''classify molecules'''
if os.path.exists("speciesNumber"):
    printf("Exist speciesNumber")
else:
    molDict = load("molDict.pkl")
    molDictDict, uqMolDict = Analyzer.classifyMol(molDict=molDict)
    Analyzer.getAllNumbFig(molDictDict=molDictDict)
    dump(uqMolDict, "uqMolDict.pkl")
    moveFiles(toDir="speciesNumber")

#[classifyMol        ] begin to classify
#[classifyMol        ] hashCode: 04e50107c20283ee38a9300940ae3055 |avgN:        0.043 |eg: H4O3
#[classifyMol        ] hashCode: 7872ef69f385c6740d64c24100ea1d2a |avgN:          5.0 |eg: C116H165N6O10
#[classifyMol        ] hashCode: eb8b8b1e5368943aa07901aa28f0af06 |avgN:       14.674 |eg: HO
#[classifyMol        ] hashCode: 8382f5c44849957831b659344b937a03 |avgN:       10.239 |eg: H3O2
#[classifyMol        ] hashCode: 77d282ba0209ea6aaf3671b6f51bf585 |avgN:      294.717 |eg: H2O
#[classifyMol        ] classfied done
#[getNumbFig         ]beginning:
#[getNumbFig         ] hashCode: 04e50107c20283ee38a9300940ae3055 |avgN:        0.043 |eg: H4O3
#[drawFigMulti       ] store as: fig-molN-H4O3-e3055.png
#[listToCSV          ] store as: raw-molN-H4O3-e3055.csv
#[getNumbFig         ] hashCode: 7872ef69f385c6740d64c24100ea1d2a |avgN:          5.0 |eg: C116H165N6O10
#[drawFigMulti       ] store as: fig-molN-C116H165N6O10-a1d2a.png
#[listToCSV          

In [8]:
''' get rex number'''
if os.path.exists("rexNumber"):
    printf("Exist rexNumber")
else:
    rexDict = load("rexDict.pkl")
    rexClassDict, uqRexDict = Analyzer.classifyRex(rexDict=rexDict)
    Analyzer.getRexNumb(rexDictDict=rexClassDict)
    dump(uqRexDict, "uqRexDict.pkl")
    moveFiles(toDir="rexNumber")

#[classifyRex        ] begin to classify the reactions
1 |rexHash: -5754912552265752145 |num: 23658 |eg: [Step:5990000-6000000|ID:281] =>  |details:  H2O => H2O 
2 |rexHash: -3481777291648637014 |num: 455 |eg: [Step:5990000-6000000|ID:32] =>  |details:  C116H165N6O10 => C116H165N6O10 
3 |rexHash: 7459992192444542910 |num: 225 |eg: [Step:5990000-6000000|ID:126] =>  |details:  HO => HO 
4 |rexHash: -7750093019039130967 |num: 215 |eg: [Step:5990000-6000000|ID:26] =>  |details:  HO + H2O => HO + H2O 
5 |rexHash: -1015104079606628526 |num: 188 |eg: [Step:5990000-6000000|ID:205] H3O2 => HO + H2O  |details:  H3O2 => HO + H2O 
6 |rexHash: -3095784518382453368 |num: 187 |eg: [Step:5980000-5990000|ID:201] H2O + HO => H3O2  |details:  H2O + HO => H3O2 
7 |rexHash: 968977637870546479 |num: 102 |eg: [Step:5990000-6000000|ID:107] HO + H2O => H3O2  |details:  H2O + HO + H2O => H3O2 + H2O 
8 |rexHash: 8208081098099381386 |num: 100 |eg: [Step:5990000-6000000|ID:99] =>  |details:  H3O2 + H2O => H3O2 + H

Draw reaction network by classified Molecules and Reactions

In [9]:
''' draw rex network'''
if os.path.exists("rexNet"):
    printf("Exist rexNet")
else:
    uqMolDict = load("uqMolDict.pkl")
    uqRexDict = load("uqRexDict.pkl")
    avgNumbList = [egMol['num'] for egMol in uqMolDict.values()]
    
    spsNumbShown = 10 # show 10 species
    if len(avgNumbList) <=spsNumbShown:
        lowestN = 0.0
    else:
        sortedAvgNumbList = sorted(avgNumbList, reverse=True)
        lowestN = sortedAvgNumbList[spsNumbShown] + 0.00001

    rexIdxDict = Analyzer.drawRexNet(uqMolDict=uqMolDict, uqRexDict=uqRexDict, lowestN=lowestN)
    dump(rexIdxDict, "rexIdxDict.pkl")
    moveFiles(toDir="rexNet", fileNames=["rexIdxDict.pkl",])

#[drawRexNet         ] begin to: draw the rexNet
#[drawRexNet         ] rexNumb : 1-3|4
#[drawRexNet         ] rexNumb : 1-4|4
#[drawRexNet         ] rexNumb : 1-5|4
#[drawRexNet         ] rexNumb : 3-4|534
#[drawRexNet         ] rexNumb : 3-5|764
#[drawRexNet         ] rexNumb : 3-1|4
#[drawRexNet         ] rexNumb : 4-3|537
#[drawRexNet         ] rexNumb : 4-5|752
#[drawRexNet         ] rexNumb : 4-1|4
#[drawRexNet         ] rexNumb : 5-3|769
#[drawRexNet         ] rexNumb : 5-4|753
#[drawRexNet         ] rexNumb : 5-1|4
#[drawRexNet         ] mol idx : 1 |symbols: H4O3 |hashCode: 04e50107c20283ee38a9300940ae3055
#[drawRexNet         ] mol idx : 2 |symbols: C116H165N6O10 |hashCode: 7872ef69f385c6740d64c24100ea1d2a
#[drawRexNet         ] mol idx : 3 |symbols: HO |hashCode: eb8b8b1e5368943aa07901aa28f0af06
#[drawRexNet         ] mol idx : 4 |symbols: H3O2 |hashCode: 8382f5c44849957831b659344b937a03
#[drawRexNet         ] mol idx : 5 |symbols: H2O |hashCode: 77d282ba0209ea6aaf3671b6f51b

The species on the node of the network can be obtained by

In [10]:
H3O2 = load("./rexNet/rexIdxDict.pkl")[2]
print(H3O2)

| ID:  137 | Type: H | Steps: 6000000 | Charge:  0.081 | Position: (  6.461, 18.394, -1.127) | Bonds: [(126, 0.913), (126, 0.913)] | 
| ID:  227 | Type: H | Steps: 6000000 | Charge:  0.149 | Position: ( 12.181, 13.053, 10.231) | Bonds: [(221, 0.954), (221, 0.954)] | 
| ID:  216 | Type: C | Steps: 6000000 | Charge: -0.114 | Position: ( 12.209,  8.703, 11.792) | Bonds: [(215, 1.363), (217, 0.961), (219, 1.418), (215, 1.363), (217, 0.961), (219, 1.418)] | 
| ID:  282 | Type: O | Steps: 6000000 | Charge: -0.469 | Position: ( 25.213, 19.451,  5.943) | Bonds: [(269, 0.702), (283, 1.119), (269, 0.702), (283, 1.119)] | 
| ID:  293 | Type: H | Steps: 6000000 | Charge:  0.135 | Position: ( 25.849, 17.029,  6.962) | Bonds: [(289, 0.949), (289, 0.949)] | 
| ID:  271 | Type: C | Steps: 6000000 | Charge: -0.115 | Position: ( 25.200, 21.936,  6.593) | Bonds: [(269, 1.436), (270, 1.399), (275, 0.949), (269, 1.436), (270, 1.399), (275, 0.949)] | 
| ID:   20 | Type: H | Steps: 6000000 | Charge:  0.170 |

In [11]:
from ase.visualize import view # need tkinter
view(H3O2.toAseAtoms())

<Popen: returncode: None args: ['/home/mll/anaconda3/bin/python', '-m', 'ase...>

Please note that the default `ase` module need `tkinter` for visualization (see https://wiki.fysik.dtu.dk/ase/install.html)

Sometimes we only concern Reactions from H3O2. Then, we can use getRexFrom to filter the reaction network

In [12]:
'''search rex '''
aimRexs = Analyzer.getRexFrom(fromMol = H3O2, rexDict=load("rexDict.pkl"), show=False)
rexClassDict, uqRexDict = Analyzer.classifyRex(rexDict=aimRexs)
Analyzer.getRexNumb(rexDictDict=rexClassDict)
moveFiles(toDir="rexs-from-" + H3O2.getSymbols())

rexIdxDict = Analyzer.drawRexNet(uqMolDict = load("uqMolDict.pkl"), uqRexDict=uqRexDict, lowestN=0)
dump(rexIdxDict, "rexIdxDict.pkl")
moveFiles(toDir="rexNet-from-" + H3O2.getSymbols(), fileNames=["rexIdxDict.pkl",])

#[getRexFrom         ] get rex from C116H165N6O10
#[classifyRex        ] begin to classify the reactions
#[classifyRex        ] classify the reactions done
#[getRexNumb         ] finished
#[moveFiles          ] moveFile: ['allRexNumb.csv']
#[drawRexNet         ] begin to: draw the rexNet
#[drawRexNet         ] mol idx : 1 |symbols: H4O3 |hashCode: 04e50107c20283ee38a9300940ae3055
#[drawRexNet         ] mol idx : 2 |symbols: C116H165N6O10 |hashCode: 7872ef69f385c6740d64c24100ea1d2a
#[drawRexNet         ] mol idx : 3 |symbols: HO |hashCode: eb8b8b1e5368943aa07901aa28f0af06
#[drawRexNet         ] mol idx : 4 |symbols: H3O2 |hashCode: 8382f5c44849957831b659344b937a03
#[drawRexNet         ] mol idx : 5 |symbols: H2O |hashCode: 77d282ba0209ea6aaf3671b6f51bf585
#[drawRexNet         ] store as: RexNet.png
#[moveFiles          ] moveFile: ['RexNet.png', 'rexIdxDict.pkl']


## 9. Drift length

In [14]:
'''get drift length'''
from RDAnalyzer.species import Mol

if os.path.exists("driftLength"):
    printf("Exist driftLength")
else:
    rexDict = load("rexDict.pkl")
    Analyzer.getDriftLength(rexDict=rexDict, targetMols=(Mol('OH'), Mol("H3O2"), Mol('H5O3')), nCores=8)
    moveFiles(toDir="driftLength")

#[getDriftLength     ] begin to: get drift length by 8 cores
#[drawFigMulti       ] store as: fig-x-MassCenter-GroVehSrf-of-Target-Species.png
#[drawFigMulti       ] store as: fig-y-MassCenter-GroVehSrf-of-Target-Species.png
#[drawFigMulti       ] store as: fig-z-MassCenter-GroVehSrf-of-Target-Species.png
#[drawFigMulti       ] store as: fig-Number-of-used-molecules-GroVeh.png
#[listToCSV          ] store as: getMassCenter_VehGroSrf_MP.csv
#[getDriftLength     ] finished: get MassCenter
#[moveFiles          ] moveFile: ['fig-Number-of-used-molecules-GroVeh.png', 'fig-x-MassCenter-GroVehSrf-of-Target-Species.png', 'fig-y-MassCenter-GroVehSrf-of-Target-Species.png', 'fig-z-MassCenter-GroVehSrf-of-Target-Species.png', 'getMassCenter_VehGroSrf_MP.csv']


Abbreviations:  
- gro: grotthuss  
- veh: vehicular  
- sol: free (diffusion in solvent)  
- srf: associated (associated with surface cationic groups)